<a href="https://colab.research.google.com/github/ErenB02/comp_bio/blob/Labs(DM)/MOCKEXAM(25%25).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS6405 - Data Mining - Dry Run

### Submission

You should submit a single .ipnyb file with your python code and analysis electronically via Canvas.
Please note that this assignment will account for 25 Marks of your module grade.

### Declaration

By submitting this assignment. I agree to the following:

<font color="red">“I have read and understand the UCC academic policy on plagiarism, and agree to the requirements set out thereby in relation to plagiarism and referencing. I confirm that I have referenced and acknowledged properly all sources used in the preparation of this assignment.
I declare that this assignment is entirely my own work based on my personal study. I further declare that I have not engaged the services of another to either assist me in, or complete this assignment”</font>




## Data Preparation
Prepare the dataset between features and labels.

In [26]:
import pandas as pd

# Load the training data
df = pd.read_csv("https://raw.githubusercontent.com/andvise/DataAnalyticsDatasets/refs/heads/main/sat_dataset_train.csv")

# Load the test data
test_df = pd.read_csv("https://raw.githubusercontent.com/andvise/DataAnalyticsDatasets/refs/heads/main/sat_dataset_test.csv")

df.head()

,c,v,clauses_vars_ratio,vars_clauses_ratio,vcg_var_mean,vcg_var_coeff,vcg_var_min,vcg_var_max,vcg_var_entropy,vcg_clause_mean,...,rwh_0_max,rwh_1_mean,rwh_1_coeff,rwh_1_min,rwh_1_max,rwh_2_mean,rwh_2_coeff,rwh_2_min,rwh_2_max,target
0,420,10,42.000000,0.023810,0.600000,0.000000,0.600000,0.600000,0.000000,0.600000,...,78750.0,0.000008,0.0,7.875000e-06,0.000008,2.385082e-21,0.0,2.385082e-21,2.385082e-21,1
1,230,20,11.500000,0.086957,0.137826,0.089281,0.117391,0.160870,2.180946,0.137826,...,6646875.0,17433.722184,1.0,2.981244e-12,34867.444369,1.727721e+04,1.0,1.358551e-53,3.455442e+04,0
2,240,16,15.000000,0.066667,0.300000,0.000000,0.300000,0.300000,0.000000,0.300000,...,500000.0,1525.878932,0.0,1.525879e+03,1525.878932,1.525879e+03,0.0,1.525879e+03,1.525879e+03,1
3,424,30,14.133333,0.070755,0.226415,0.485913,0.056604,0.452830,2.220088,0.226415,...,87500.0,0.000122,1.0,6.535723e-14,0.000245,8.218628e-07,1.0,1.499676e-61,1.643726e-06,0
4,162,19,8.526316,0.117284,0.139701,0.121821,0.111111,0.185185,1.940843,0.139701,...,5859400.0,16591.494310,1.0,6.912726e-42,33182.988621,1.665903e+04,1.0,0.000000e+00,3.331807e+04,1


In [27]:
import numpy as np

# Replace NaN values with 0 and infinite values with float max
df.fillna(0, inplace=True)
df.replace([np.inf, -np.inf], np.finfo(np.float16).max, inplace=True)

# Do the same for the test_df
test_df.fillna(0, inplace=True)
test_df.replace([np.inf, -np.inf], np.finfo(np.float16).max, inplace=True)

In [28]:
df.dtypes

,0
c,int64
v,int64
clauses_vars_ratio,float64
vars_clauses_ratio,float64
vcg_var_mean,float64
...,...
rwh_2_mean,float64
rwh_2_coeff,float64
rwh_2_min,float64
rwh_2_max,float64


In [13]:
print(df['target'].value_counts())
#Proportion of value counts in target (alternative for below)
print(df['target'].value_counts(normalize=True) * 100)

print(test_df['target'].value_counts())
print(test_df['target'].value_counts(normalize=True) * 100)


target
1    976
0    953
Name: count, dtype: int64
target
1    50.596164
0    49.403836
Name: proportion, dtype: float64
target
1    245
0    238
Name: count, dtype: int64
target
1    50.724638
0    49.275362
Name: proportion, dtype: float64


IS the train test split stratified?

In [9]:
# YOUR CODE HERE
print(sum(df['target'] == 1)/(df.shape[0]) * 100)
print(sum(test_df['target'] == 1)/(test_df.shape[0]) * 100)

50.59616381544841
50.72463768115942


- The train test split is stratified as observed above. The target (class) variable in either train and test data shows that the encoded variable (0,1) is evenly split with 50%.

## Machine Learning Pipeline

### Part 1
Develop a machine learning pipeline that given the statistical features of a CNF predicts if the problem is satisfiable or not. Consider which components of a pipeline to include and briefly justify your choice.

Conduct a brief hyperparametrization.


In [29]:
# YOUR CODE HERE

print(df["target"].shape)
print(test_df["target"].shape)
#The dimensions of the variables show lots of instances thus dimensionality reduction would be a good idea

from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

#Want to normalize our values as we observe that some range from ... to .. thus we do this to avoid outliers from affecting our data
#Here we use a classifier as the task asks that we classify either satisfiable or non-satisfiable
classifierKNN = Pipeline([
    ("scaler", MinMaxScaler()),
    ("predictor", KNeighborsClassifier())
])


#Set a parameter grid referencing the pipeline
#Set range of k neighbours
knn_param_grid = {
    "predictor__n_neighbors": [3,5,7,9,10,15,20,25,30],
    "predictor__p": [1, 2]}


# Create the GridSearchCV object which will search for the best hyperparameters based on cross-validated accuracy.
knn_gs = GridSearchCV(classifierKNN, knn_param_grid, scoring="accuracy")

# Fit the model with the training data to find the best hyperparameters.
knn_gs.fit(df.loc[:, :"rwh_2_max"], df["target"])

# Print the best hyperparameters and the best score found during the grid search.
knn_gs.best_params_, knn_gs.best_score_




(1929,)
(483,)


({'predictor__n_neighbors': 7, 'predictor__p': 1}, 0.9160325684678016)

### Part 2
Evaluate your model in the test set.

Is the accuracy you obtained higher than that of a baseline classifier?

In [37]:
#Here we set a baseline classifier as a reference point for our model seen above
#Thus any test accuracy in the model that is lower than this refernce percentage indicates poor accuracy
print((test_df["target"] == 0).mean())
print((test_df["target"] == 1).mean())


# Set the best hyperparameters from GridSearchCV into the classifier pipeline
#TAKEN FROM GRID SEARCH ABOVE
classifierKNN.set_params(**knn_gs.best_params_)
# Fit the pipeline with the training data using the best parameters found
classifierKNN.fit(df.loc[:, :"rwh_2_max"], df["target"])
# Calculate accuracy on the test set
accuracy_score(test_df["target"], classifierKNN.predict(test_df.loc[:, :"rwh_2_max"]))

0.4927536231884058
0.5072463768115942


0.9130434782608695